In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.initializers import Constant
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Definir la restricción de ortogonalidad para los pesos
class OrthogonalConstraint(Constraint):
    def __call__(self, w):
        a, u, _ = tf.linalg.svd(w, full_matrices=False)
        return u

# Definir el autoencoder lineal
class LinearAutoencoder(tf.keras.Model):
    def __init__(self, input_dim, encoding_dim):
        super(LinearAutoencoder, self).__init__()
        self.input_dim = input_dim
        self.encoding_dim = encoding_dim

        initial_weight = np.ones((input_dim, encoding_dim))
        q, _ = np.linalg.qr(initial_weight)  # QR decomposition for orthogonal matrix

        self.dense_layer = tf.keras.layers.Dense(encoding_dim, use_bias=False,
                                                 kernel_initializer=Constant(q),
                                                 kernel_constraint=OrthogonalConstraint())

    def call(self, inputs):
        encoded = self.dense_layer(inputs)
        decoded = tf.matmul(encoded, self.dense_layer.kernel, transpose_b=True)
        return decoded

    # Method to get the encoded representation
    def encode(self, inputs):
        return self.dense_layer(inputs)


# Generar datos sintéticos
np.random.seed(42)
x_train = np.dot(np.random.random(size=(1000, 2)), np.random.random((2, 3))).astype(np.float32)
x_train = x_train
x_train = StandardScaler().fit_transform(x_train)  # Estandarizar los datos

# PCA convencional
pca = PCA(n_components=2)
x_train_pca = pca.fit_transform(x_train)

# Autoencoder lineal
autoencoder = LinearAutoencoder(input_dim=3, encoding_dim=2)
autoencoder.compile(optimizer=tf.keras.optimizers.SGD(1e-3), loss='mse')
autoencoder.fit(x_train, x_train, epochs=100, batch_size=32, verbose=0)

# Extraer los componentes aprendidos por el autoencoder
weights = autoencoder.dense_layer.get_weights()[0]  # Pesos de la capa densa
encoded_data = autoencoder.encode(x_train)
# Comparar visualmente PCA convencional y autoencoder
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear figuras subplots interactivas con Plotly
fig = make_subplots(rows=1, cols=2, subplot_titles=("PCA Convencional", "Autoencoder Lineal"))

# Agregar el gráfico de PCA al primer subplot
fig.add_trace(
    go.Scatter(x=x_train_pca[:, 0], y=x_train_pca[:, 1], mode='markers', name='PCA'),
    row=1, col=1
)

# Agregar el gráfico del autoencoder al segundo subplot
fig.add_trace(
    go.Scatter(x=encoded_data[:, 0], y=encoded_data[:, 1], mode='markers', name='Autoencoder'),
    row=1, col=2
)

# Actualizar los títulos de los ejes para el subplot de PCA
fig.update_xaxes(title_text="Componente Principal 1", row=1, col=1)
fig.update_yaxes(title_text="Componente Principal 2", row=1, col=1)

# Actualizar los títulos de los ejes para el subplot del Autoencoder
fig.update_xaxes(title_text="Característica aprendida 1", row=1, col=2)
fig.update_yaxes(title_text="Característica aprendida 2", row=1, col=2)

# Actualizar el layout para mejorar la visualización
fig.update_layout(height=600, width=800, title_text="Comparación PCA vs Autoencoder Lineal")
fig.show()

# Ortogonalidad

In [2]:
weights.T@weights,pca.components_@pca.components_.T

(array([[ 1.0000000e+00, -1.8175049e-08],
        [-1.8175049e-08,  1.0000001e+00]], dtype=float32),
 array([[ 1.0000002e+00, -5.1873243e-08],
        [-5.1873243e-08,  1.0000002e+00]], dtype=float32))